In [39]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import torch
from facenet_pytorch import MTCNN, InceptionResnetV1
from sklearn.metrics import accuracy_score, roc_curve, auc

In [40]:
lfw_dir = 'dataset/lfw_funneled'
pairs_file = 'dataset/pairsDevTest.txt'

with open(pairs_file, 'r') as f:
    lines = f.readlines()[1:]

pairs = []
labels = []

for line in lines:
    parts = line.strip().split()
    if len(parts) == 3:
        # positive pair: name, idx1, idx2
        name, idx1, idx2 = parts
        img1 = os.path.join(lfw_dir, name, f"{name}_{int(idx1):04d}.jpg")
        img2 = os.path.join(lfw_dir, name, f"{name}_{int(idx2):04d}.jpg")
        label = 1
    elif len(parts) == 4:
        # negative pair: name1, idx1, name2, idx2
        name1, idx1, name2, idx2 = parts
        img1 = os.path.join(lfw_dir, name1, f"{name1}_{int(idx1):04d}.jpg")
        img2 = os.path.join(lfw_dir, name2, f"{name2}_{int(idx2):04d}.jpg")
        label = 0
    else:
        print(f"Invalid line: {parts}")
        break

    if os.path.exists(img1) and os.path.exists(img2):
        pairs.append((img1, img2))
        labels.append(label)
    else:
        print(f"Invalid line: {parts}")
        break

pair_labels = [(pairs[i], labels[i]) for i in range(len(pairs))]
print(f"Загружено {len(pairs)} пар")

Загружено 1000 пар


In [41]:
print(torch.cuda.is_available())
device = 'cuda' if torch.cuda.is_available() else 'cpu'
mtcnn = MTCNN(image_size=160, margin=0, device=device)
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

False


In [ ]:
def get_embedding(img_path, show=False):
    img = Image.open(img_path).convert('RGB')
    face = mtcnn(img)

    if face is None:
        print("Лицо не обнаружено.")
        return None

    if show:
        # Переводим тензор в numpy и транспонируем под формат [H, W, C]
        face_np = face.permute(1, 2, 0).cpu().numpy()
        face_np = np.clip(face_np, 0, 1)  # на случай значений вне диапазона

        # Отображение
        fig, axs = plt.subplots(1, 2, figsize=(8, 4))
        axs[0].imshow(img)
        axs[0].set_title("Оригинал")
        axs[0].axis('off')

        axs[1].imshow(face_np)
        axs[1].set_title("Выровненное лицо")
        axs[1].axis('off')

        plt.tight_layout()
        plt.show()

    with torch.no_grad():
        embedding = model(face.unsqueeze(0).to(device))
    return embedding.cpu().numpy()[0]

In [43]:
embeddings1, embeddings2, y_true = [], [], []
for (img1, img2), label in tqdm(pair_labels, total=len(pair_labels)):
    e1 = get_embedding(img1)
    e2 = get_embedding(img2)
    # Если не нашли лицо в одном из изображений — пропускаем пару
    if e1 is None or e2 is None:
        continue
    embeddings1.append(e1)
    embeddings2.append(e2)
    y_true.append(label)

embeddings1 = np.vstack(embeddings1)
embeddings2 = np.vstack(embeddings2)
y_true = np.array(y_true)

  3%|▎         | 26/1000 [00:04<03:03,  5.32it/s]


KeyboardInterrupt: 

In [ ]:
distances = np.linalg.norm(embeddings1 - embeddings2, axis=1)

# 4. Определяем оптимальный порог по ROC-кривой
fpr, tpr, thresholds = roc_curve(y_true, -distances)  
# (отрицание, потому что более похожие — меньшее расстояние)
roc_auc = auc(fpr, tpr)

# Выбираем threshold, при котором tpr – (1 – fpr) максимально
gmeans = np.sqrt(tpr * (1 - fpr))
ix = np.argmax(gmeans)
best_thresh = thresholds[ix]

# 5. Оцениваем точность при этом пороге
y_pred = (distances < -best_thresh).astype(int)
acc = accuracy_score(y_true, y_pred)

print(f"AUC = {roc_auc:.4f}")
print(f"Optimal threshold = { -best_thresh :.4f}")
print(f"Accuracy = {acc:.4f}")

AUC = 0.9751
Optimal threshold = 1.0212
Accuracy = 0.9650


In [58]:
def verify_faces(img1_path, img2_path, threshold=1.0):
    """
    Computes the Euclidean distance between two face embeddings
    and compares it to the threshold.
    """
    emb1 = get_embedding(img1_path)
    emb2 = get_embedding(img2_path)
    dist = np.linalg.norm(emb1 - emb2)
    print(f"Distance: {dist:.4f}")
    if dist < threshold:
        print("✅ Same person")
    else:
        print("❌ Different persons")

In [ ]:
sample = 'dataset/lfw_funneled/Donald_Trump/Donald_Trump_0001.jpg'

image0 = 'dataset/lfw_funneled/Donald_Trump/Donald_Trump_0001.jpg'
image1 = 'dataset/lfw_funneled/Princess_Aiko/Princess_Aiko_0002.jpg'
image2 = 'dataset/lfw_funneled/Zico/Zico_0001.jpg'
image3 = 'dataset/lfw_funneled/Princess_Caroline/Princess_Caroline_0001.jpg'
image4 = 'dataset/lfw_funneled/Princess_Diana/Princess_Diana_0001.jpg'
image5 = 'dataset/lfw_funneled/Princess_Hisako/Princess_Hisako_0001.jpg'

verify_faces(sample, image0)
verify_faces(sample, image1)
verify_faces(sample, image2)
verify_faces(sample, image3)
verify_faces(sample, image4)
verify_faces(sample, image5)

Distance: 1.3063
❌ Different persons
Distance: 1.2539
❌ Different persons
Distance: 1.3976
❌ Different persons
Distance: 1.4313
❌ Different persons
Distance: 1.2731
❌ Different persons
